## Testing with one URL

In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = "https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/"

In [3]:
page = requests.get(url)

In [5]:
soup = BeautifulSoup(page.content,"lxml")

In [17]:
title = soup.find("h1",class_="entry-title").text

In [16]:
article = soup.find("div",class_="td-pb-span8 td-main-content").text.strip().replace("\n","")

In [21]:
urls = pd.read_csv("Input.xlsx - Sheet1.csv")
urls

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...
...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...


## Applying model to each URL

In [36]:
data = []

In [37]:
def extract_data(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content,"lxml")
    title = soup.find("h1",class_="entry-title")
    if title:
        title = title.text
    else:
        title = "title not found"
    article = soup.find("div",class_="td-pb-span8 td-main-content")
    if article:
        article = article.text.strip().replace("\n","")
    else:
        article = "article not found"
    data.append((title,article))
    

In [38]:
for i in urls["URL"]:
    extract_data(str(i))

In [189]:
df = pd.DataFrame(data)

In [190]:
df.columns = ["title","article"]

In [191]:
mask = df["title"]=="title not found"

In [192]:
df = df[~mask]

In [201]:
df

,title,article,article_s
0,Rising IT cities and its impact on the economy...,We have seen a huge development and dependence...,[We have seen a huge development and dependenc...
1,Rising IT Cities and Their Impact on the Econo...,"Throughout history, from the industrial revolu...","[Throughout history, from the industrial revol..."
2,"Internet Demand’s Evolution, Communication Imp...","IntroductionIn the span of just a few decades,...",[IntroductionIn the span of just a few decades...
3,Rise of Cybercrime and its Effect in upcoming ...,"The way we live, work, and communicate has unq...","[The way we live, work, and communicate has un..."
4,OTT platform and its impact on the entertainme...,The year 2040 is poised to witness a continued...,[The year 2040 is poised to witness a continue...
...,...,...,...
93,Gaming Disorder and Effects of Gaming on Health.,Perhaps the virtual illusion has become today’...,[Perhaps the virtual illusion has become today...
94,What is the repercussion of the environment du...,What is COVID 19 pandemic?On 31st December 201...,[What is COVID 19 pandemic?On 31st December 20...
95,Due to the COVID-19 the repercussion of the en...,"Epidemics, in general, have both direct and in...","[Epidemics, in general, have both direct and i..."
96,Impact of COVID-19 pandemic on office space an...,COVID 19 has bought the world to its knees. Wi...,"[COVID 19 has bought the world to its knees., ..."


In [197]:
df["article_s"]= df["article"].apply(sent_tokenize)

In [204]:
df

,title,article,article_s
0,Rising IT cities and its impact on the economy...,We have seen a huge development and dependence...,[We have seen a huge development and dependenc...
1,Rising IT Cities and Their Impact on the Econo...,Throughout history from the industrial revolu...,"[Throughout history, from the industrial revol..."
2,"Internet Demand’s Evolution, Communication Imp...",IntroductionIn the span of just a few decades ...,[IntroductionIn the span of just a few decades...
3,Rise of Cybercrime and its Effect in upcoming ...,The way we live work and communicate has unq...,"[The way we live, work, and communicate has un..."
4,OTT platform and its impact on the entertainme...,The year is poised to witness a continued...,[The year 2040 is poised to witness a continue...
...,...,...,...
93,Gaming Disorder and Effects of Gaming on Health.,Perhaps the virtual illusion has become today ...,[Perhaps the virtual illusion has become today...
94,What is the repercussion of the environment du...,What is COVID pandemic On st December ...,[What is COVID 19 pandemic?On 31st December 20...
95,Due to the COVID-19 the repercussion of the en...,Epidemics in general have both direct and in...,"[Epidemics, in general, have both direct and i..."
96,Impact of COVID-19 pandemic on office space an...,COVID has bought the world to its knees Wi...,"[COVID 19 has bought the world to its knees., ..."


In [146]:
import re

In [202]:
def filtering_text(text):
    text = re.sub("[^a-zA-Z]"," ", text)
    return text

In [203]:
df["article"] = df["article"].apply(lambda x : filtering_text(x))

In [71]:
file1 = open("StopWords_Auditor.txt", "r")
file2 = open("StopWords_Currencies.txt", "r")
file3 = open("StopWords_DatesandNumbers.txt", "r")
file4 = open("StopWords_Generic.txt", "r")
file5 = open("StopWords_GenericLong.txt", "r")
file6 = open("StopWords_Geographic.txt", "r")
file7 = open("StopWords_Names.txt", "r")

output_file = open("stopwords.txt", "w")

output_file.write(file1.read())
output_file.write(file2.read())
output_file.write(file3.read())
output_file.write(file4.read())


file1.close()
file2.close()
file3.close()
file4.close()
output_file.close()


In [205]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

In [206]:
with open("stopwords.txt", "r") as file:
    stopWords = file.read().splitlines()

def removing_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stopWords]
    return filtered_text



In [207]:
stopWords = [x.lower() for x in stopWords]

In [208]:
stopWords

['ernst',
 'young',
 'deloitte',
 'touche',
 'kpmg',
 'pricewaterhousecoopers',
 'pricewaterhouse',
 'coopers',
 'afghani  | afghanistan ',
 'ariary | madagascar ',
 'baht | thailand ',
 'balboa | panama ',
 'birr | ethiopia ',
 'bolivar | venezuela ',
 'boliviano  | bolivia ',
 'cedi | ghana ',
 'colon  | costa rica ',
 'córdoba  | nicaragua ',
 'dalasi | gambia ',
 'denar | macedonia (former yug. rep.) ',
 'dinar | algeria ',
 'dirham  | morocco ',
 'dobra | são tom and príncipe ',
 'dong | vietnam ',
 'dram | armenia ',
 'escudo  | cape verde ',
 'euro  | belgium ',
 'florin | aruba ',
 'forint | hungary ',
 'gourde | haiti ',
 'guarani | paraguay ',
 'gulden | netherlands antilles ',
 'hryvnia  | ukraine ',
 'kina | papua new guinea ',
 'kip | laos ',
 'konvertibilna marka  | bosnia-herzegovina ',
 'koruna  | czech republic ',
 'krona | sweden ',
 'krone | denmark ',
 'kroon | estonia ',
 'kuna | croatia ',
 'kwacha | zambia ',
 'kwanza | angola ',
 'kyat | myanmar ',
 'lari | geor

In [209]:
df["article"] = df["article"].apply(lambda x : removing_stopwords(x))

In [211]:
df

,title,article,article_s
0,Rising IT cities and its impact on the economy...,"[seen, a, huge, development, dependence, peopl...",[We have seen a huge development and dependenc...
1,Rising IT Cities and Their Impact on the Econo...,"[Throughout, history, industrial, revolution, ...","[Throughout history, from the industrial revol..."
2,"Internet Demand’s Evolution, Communication Imp...","[IntroductionIn, span, a, decades, internet, u...",[IntroductionIn the span of just a few decades...
3,Rise of Cybercrime and its Effect in upcoming ...,"[way, live, work, communicate, unquestionably,...","[The way we live, work, and communicate has un..."
4,OTT platform and its impact on the entertainme...,"[poised, witness, a, continued, revolution, wo...",[The year 2040 is poised to witness a continue...
...,...,...,...
93,Gaming Disorder and Effects of Gaming on Health.,"[Perhaps, virtual, illusion, become, today, s,...",[Perhaps the virtual illusion has become today...
94,What is the repercussion of the environment du...,"[COVID, pandemic, st, a, novel, coronavirus, i...",[What is COVID 19 pandemic?On 31st December 20...
95,Due to the COVID-19 the repercussion of the en...,"[Epidemics, general, direct, indirect, costs, ...","[Epidemics, in general, have both direct and i..."
96,Impact of COVID-19 pandemic on office space an...,"[COVID, bought, world, knees, businesses, shut...","[COVID 19 has bought the world to its knees., ..."


In [212]:
with open("positive-words.txt", "r") as file:
    positiveWords = file.read().splitlines()
with open("negative-words.txt", "r") as file:
    negtiveWords = file.read().splitlines()

In [213]:
def positiveScore(listWords):
    positive_score = 0
    for i in listWords:
        if i in positiveWords:
            positive_score+=1
    return positive_score

In [214]:
positive_score=df["article"].apply(lambda x : positiveScore(x) )

In [215]:
positive_score

0     45
1     64
2     40
3     38
4     23
      ..
93    30
94    11
95    29
96    31
97     7
Name: article, Length: 89, dtype: int64

In [216]:
def negativeScore(listWords):
    negative_score = 0
    for i in listWords:
        if i in negtiveWords:
            negative_score+=1
    return negative_score

In [217]:
negative_score = df["article"].apply(lambda x : negativeScore(x))

In [218]:
negative_score

0      5
1     31
2     25
3     71
4      9
      ..
93    44
94    25
95    55
96    34
97     2
Name: article, Length: 89, dtype: int64

In [219]:
polarity_score =(positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)

In [220]:
polarity_score

0     0.800000
1     0.347368
2     0.230769
3    -0.302752
4     0.437500
        ...   
93   -0.189189
94   -0.388889
95   -0.309524
96   -0.046154
97    0.555555
Name: article, Length: 89, dtype: float64

In [221]:
subjectivity_score = (positive_score + negative_score)/ ((df["article"].apply(len)) + 0.000001)

In [222]:
average_sentence_length = df["article"].apply(len) / df["article_s"].apply(len)

In [223]:
average_sentence_length

0     12.350000
1     14.515152
2     16.170213
3     19.025641
4     17.535714
        ...    
93    12.351852
94    20.842105
95    15.062500
96    22.500000
97    23.833333
Length: 89, dtype: float64

In [224]:
def complexWords(text):
    count = 0
    for word in text:
        vowel_count = 0
        for char in word:
            if char.lower() in 'aeiou':
                vowel_count += 1
        if vowel_count > 2:
            count += 1
    return count


In [225]:
complex_words = df["article"].apply(lambda x : complexWords(x))

In [226]:
complex_words

0     287
1     513
2     449
3     420
4     221
     ... 
93    281
94    178
95    330
96    247
97    144
Name: article, Length: 89, dtype: int64

In [230]:
Percentage_of_Complex_words = (complex_words / df["article"].apply(len))*100

In [231]:
Percentage_of_Complex_words

0     38.731444
1     53.549061
2     59.078947
3     56.603774
4     45.010183
        ...    
93    42.128936
94    44.949495
95    45.643154
96    39.206349
97    50.349650
Name: article, Length: 89, dtype: float64

In [233]:
fog_Index = 0.4 * (average_sentence_length + Percentage_of_Complex_words)

In [234]:
fog_Index

0     20.432578
1     27.225685
2     30.099664
3     30.251766
4     25.018359
        ...    
93    21.792315
94    26.316640
95    24.282261
96    24.682540
97    29.673193
Length: 89, dtype: float64

In [235]:
word_count = df["article"].apply(len)

In [240]:
import syllables

In [262]:
def syllableCount(word):
    count = 0
    for w in word:
        count += syllables.estimate(w)
    return count

In [264]:
df["Syllable Count"] = df["article_s"].apply(lambda x: syllableCount(x))

In [356]:
syllable_count_per_word = df["Syllable Count"]/ word_count

In [357]:
df

,title,article,article_s,Syllable Count,syllable_count_per_word
0,Rising IT cities and its impact on the economy...,"[seen, a, huge, development, dependence, peopl...",[We have seen a huge development and dependenc...,2171,2.929825
1,Rising IT Cities and Their Impact on the Econo...,"[Throughout, history, industrial, revolution, ...","[Throughout history, from the industrial revol...",3075,3.209812
2,"Internet Demand’s Evolution, Communication Imp...","[IntroductionIn, span, a, decades, internet, u...",[IntroductionIn the span of just a few decades...,2550,3.355263
3,Rise of Cybercrime and its Effect in upcoming ...,"[way, live, work, communicate, unquestionably,...","[The way we live, work, and communicate has un...",2403,3.238544
4,OTT platform and its impact on the entertainme...,"[poised, witness, a, continued, revolution, wo...",[The year 2040 is poised to witness a continue...,1449,2.951120
...,...,...,...,...,...
93,Gaming Disorder and Effects of Gaming on Health.,"[Perhaps, virtual, illusion, become, today, s,...",[Perhaps the virtual illusion has become today...,2014,3.019490
94,What is the repercussion of the environment du...,"[COVID, pandemic, st, a, novel, coronavirus, i...",[What is COVID 19 pandemic?On 31st December 20...,1216,3.070707
95,Due to the COVID-19 the repercussion of the en...,"[Epidemics, general, direct, indirect, costs, ...","[Epidemics, in general, have both direct and i...",2234,3.089903
96,Impact of COVID-19 pandemic on office space an...,"[COVID, bought, world, knees, businesses, shut...","[COVID 19 has bought the world to its knees., ...",1907,3.026984


In [323]:
pattern=r'\b(?:I|me|my|mine|myself|you|your|yours|yourself|he|him|his|himself|she|her|hers|herself|it|its|itself|we|us|our|ours|ourselves|you|your|yours|yourselves|they|them|their|theirs|themselves)\b'


def personalPronouns(text):
    for tt in text:
        matches = re.findall(pattern, tt, flags=re.IGNORECASE)
    return len(matches)

In [324]:
personal_pronouns_count = df["article_s"].apply(lambda x : personalPronouns(x))

In [325]:
personal_pronouns_count

0     3
1     1
2     3
3     2
4     2
     ..
93    2
94    2
95    2
96    2
97    2
Name: article_s, Length: 89, dtype: int64

In [345]:
def characterCount(text):
    char = list(text)
    count_char = sum(len(word) for word in char)
    return count_char

In [346]:
character_count = df["article"].apply(lambda x : characterCount(x))

In [349]:
character_count

0     4458
1     6664
2     5856
3     5590
4     3351
      ... 
93    4166
94    2576
95    4857
96    3834
97    1982
Name: article, Length: 89, dtype: int64

In [350]:
average_word_length = character_count/word_count

In [351]:
average_word_length

0     6.016194
1     6.956159
2     7.705263
3     7.533693
4     6.824847
        ...   
93    6.245877
94    6.505051
95    6.717842
96    6.085714
97    6.930070
Name: article, Length: 89, dtype: float64

In [413]:
data = pd.DataFrame(data={"POSITIVE SCORE": positive_score,
                         "NEGATIVE SCORE": negative_score,
                         "POLARITY SCORE": polarity_score,
                         "SUBJECTIVITY SCORE": subjectivity_score,
                         "AVG SENTENCEN LENGTH":average_sentence_length,
                         "PERCENTAGE OF COMPLEX WORDS":Percentage_of_Complex_words,
                         "FOG INDEX":Fog_Index,
                         "AVG NUMBER OF WORDS PER SENTENCE":average_sentence_length,
                         "COMPLEX WORD COUNT":complex_words,
                         "WORD COUNT":word_count,
                         "SYLLABLE PER WORD":syllable_count_per_word,
                         "PERSONAL PRONOUNS":personal_pronouns_count,
                         "AVG WORD LENGTH":average_word_length})

In [414]:
data

,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCEN LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,45,5,0.800000,0.067476,12.350000,38.731444,20.432578,12.350000,287,741,2.929825,3,6.016194
1,64,31,0.347368,0.099165,14.515152,53.549061,27.225685,14.515152,513,958,3.209812,1,6.956159
2,40,25,0.230769,0.085526,16.170213,59.078947,30.099664,16.170213,449,760,3.355263,3,7.705263
3,38,71,-0.302752,0.146900,19.025641,56.603774,30.251766,19.025641,420,742,3.238544,2,7.533693
4,23,9,0.437500,0.065173,17.535714,45.010183,25.018359,17.535714,221,491,2.951120,2,6.824847
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,30,44,-0.189189,0.110945,12.351852,42.128936,21.792315,12.351852,281,667,3.019490,2,6.245877
94,11,25,-0.388889,0.090909,20.842105,44.949495,26.316640,20.842105,178,396,3.070707,2,6.505051
95,29,55,-0.309524,0.116183,15.062500,45.643154,24.282261,15.062500,330,723,3.089903,2,6.717842
96,31,34,-0.046154,0.103175,22.500000,39.206349,24.682540,22.500000,247,630,3.026984,2,6.085714


## The data was not found for 11 links so replaced with mean values

In [415]:
import numpy as np

In [416]:
data = data.reindex(list(range(df.index.min(),df.index.max()+3)),fill_value=0)

In [417]:
data = data.replace(0,data.mean())

In [418]:
data.insert(0,"URL_ID",urls["URL_ID"])

In [419]:
data.insert(1,"URL",urls["URL"])

In [420]:
data

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCEN LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,45.00,5.00,0.800000,0.067476,12.350000,38.731444,20.432578,12.350000,287.00,741.00,2.929825,3.00,6.016194
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,64.00,31.00,0.347368,0.099165,14.515152,53.549061,27.225685,14.515152,513.00,958.00,3.209812,1.00,6.956159
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,40.00,25.00,0.230769,0.085526,16.170213,59.078947,30.099664,16.170213,449.00,760.00,3.355263,3.00,7.705263
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,38.00,71.00,-0.302752,0.146900,19.025641,56.603774,30.251766,19.025641,420.00,742.00,3.238544,2.00,7.533693
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,23.00,9.00,0.437500,0.065173,17.535714,45.010183,25.018359,17.535714,221.00,491.00,2.951120,2.00,6.824847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,29.00,55.00,-0.309524,0.116183,15.062500,45.643154,24.282261,15.062500,330.00,723.00,3.089903,2.00,6.717842
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,31.00,34.00,-0.046154,0.103175,22.500000,39.206349,24.682540,22.500000,247.00,630.00,3.026984,2.00,6.085714
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,7.00,2.00,0.555555,0.031469,23.833333,50.349650,29.673193,23.833333,144.00,286.00,3.013986,2.00,6.930070
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,36.43,24.22,0.238236,0.080348,16.557173,41.289233,23.138563,16.557173,310.58,666.94,2.777108,1.79,5.917791


In [421]:
data.to_csv("AssignmentBlackOffer.csv")